# Correr experimentos
En este archivo está el código para correr los experimentos y escribir los resultados en archivos CSV.
> Los archivos se guardan en la carpeta _resultados_.

In [1]:
import math, subprocess
import pandas as pd
import numpy as np
from IPython.display import display, clear_output

A continuación leemos los datasets en dataframes de Pandas.

In [9]:
def leer_instancia(path_instancia):
    with open(path_instancia, "r") as f:
        return f.read();

In [8]:
df_mejor_caso_bt_f = pd.read_csv("instancias/mejor-caso-bt-f/indice.csv");


In [11]:
df_dinamica = pd.read_csv("instancias/dinamica/indice.csv");

In [80]:
df_bt= pd.read_csv("instancias/bt/indice.csv");

In [13]:
df_fbms = pd.read_csv("instancias/muchas-soluciones-validas/indice.csv");
df_fbps = pd.read_csv("instancias/pocas-soluciones-validas/indice.csv");


In [14]:
df_fb2 = pd.read_csv("instancias/fb2/indice.csv");

La siguiente función sirve para correr el código sobre una instancia ejecutando un método en particular.
- FB: Fuerza bruta
- BT: Backtracking con ambas podas.
- BT-F: Backtracking solamente con poda por factibilidad.
- BT-O: Backtracking solamente con poda por optimalidad.
- PD: Programación dinámica.

In [15]:
def correr_experimento(metodo, archivo_instancia):
    # Leer archivo de la instancia.
    instancia = leer_instancia(archivo_instancia)
    
    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen(["../main", metodo], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Poner la instancia en la entrada estandar.
    process.stdin.write(instancia)
    process.stdin.flush()

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0: raise(F"Hubo un error en la experimentacion para el algoritmo: {algoritmo} con la instancia {archivo_instancia}.")
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    tiempo_de_ejecucion = float(process.stderr.read());
    
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();
    
    return tiempo_de_ejecucion;

## Corremos los experimentos
Vamos a guardar una tabla con las ejecuciones y sus respectivos tiempos.

In [106]:
experimentos = [];

## Experimento Fuerza Bruta : R variable para distintos ns fijos

In [36]:
for i in range(0, df_fb2.shape[0]):
    fila = df_fb2.iloc[i];
    experimentos.append([fila["dataset"], fila["n"], fila["R"], "FB", fila["archivo"]]);

## Experimento Fuerza Bruta : Pocas/Muchas soluciones

In [37]:
for i in range(0, df_fbms.shape[0]):
    fila = df_fbms.iloc[i];
    experimentos.append([fila["dataset"], fila["n"], fila["R"], "FB", fila["archivo"]]);
    
for i in range(0, df_fbps.shape[0]):
    fila = df_fbps.iloc[i];
    experimentos.append([fila["dataset"], fila["n"], fila["R"], "FB", fila["archivo"]]);

## Experimento Back Tracking - Todos Rompen
Correr BT-F para el caso en que todos los productos rompen el jambotubo.

In [42]:
for n in range(0, df_mejor_caso_bt_f.shape[0]):
    fila_n = df_mejor_caso_bt_f.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["R"], "BT-F", fila_n["archivo"]]);

## Experimento Campana de Back Tracking

In [84]:
for i in range(0, df_bt.shape[0]):
    fila = df_bt.iloc[i];
    experimentos.append([fila["dataset"], fila["n"], fila["R"], "BT", fila["archivo"]]);

## Experimento Programacion Dinamica


In [55]:
for i in range(0, df_dinamica.shape[0]):
    fila = df_dinamica.iloc[i];
    experimentos.append([fila["dataset"], fila["n"], fila["R"], "PD", fila["archivo"]]);  

## Experimento 1 :BT vs. PD

In [97]:
Rs_max = [235,
 223,
 79,
 169,
 47,
 153,
 51,
 15,
 19,
 119,
 27,
 29,
 35,
 43,
 55,
 73,
 65,
 73,
 77,
 93,
 87,
 95,
 107,
 119,
 119,
 145,
 169,
 151,
 183,
 191]

In [99]:
for i in range(0, df_bt.shape[0]):
    fila = df_bt.iloc[i];
    n = fila["n"] 
    R = fila["R"]
    if (Rs_max[n - 2]  == R):
        experimentos.append([fila["dataset"], fila["n"], fila["R"], "PD", fila["archivo"]]);

## Experimento 2 :BT vs. PD

In [107]:
for i in range(0, df_fbps.shape[0]):
    fila = df_fbps.iloc[i];
    experimentos.append([fila["dataset"], fila["n"], fila["R"], "PD", fila["archivo"]]);
    experimentos.append([fila["dataset"], fila["n"], fila["R"], "BT", fila["archivo"]]);    

## Ejecutar los experimentos y guardar los resultados en un archivo CSV.
Este paso puede tardar unos minutos hasta terminar de ejecutarse.

In [108]:
columnas = ["dataset", "n", "R", "metodo", "tiempo"];
filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiempos = []
    resultados = []
    for i in range(0, T):
        t = correr_experimento(experimento[3], experimento[4])
        #t = exp[0]
        #res = exp[1]
        #resultados.append(res)
        tiempos.append(t);
    tiempo = np.median(tiempos);
    #print(resultados)
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], tiempo]);


'Experimento: 48/48'

In [109]:
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado

,dataset,n,R,metodo,tiempo
0,pocas-soluciones-validas,1,1000,PD,0.007021
1,pocas-soluciones-validas,1,1000,BT,0.001080
2,pocas-soluciones-validas,2,1000,PD,0.007217
3,pocas-soluciones-validas,2,1000,BT,0.000984
4,pocas-soluciones-validas,3,1000,PD,0.008348
5,pocas-soluciones-validas,3,1000,BT,0.001006
6,pocas-soluciones-validas,4,1000,PD,0.009688
7,pocas-soluciones-validas,4,1000,BT,0.001092
8,pocas-soluciones-validas,5,1000,PD,0.010920
9,pocas-soluciones-validas,5,1000,BT,0.001071


In [110]:
df_aguardar = pd.read_csv("resultados/resultado.csv");
df_aguardar 

,dataset,n,R,metodo,tiempo
0,fb2,5,1,FB,0.002025
1,fb2,5,6,FB,0.001162
2,fb2,5,11,FB,0.001282
3,fb2,5,16,FB,0.001511
4,fb2,5,21,FB,0.001092
...,...,...,...,...,...
6169,bt,27,145,PD,0.031709
6170,bt,28,169,PD,0.034329
6171,bt,29,151,PD,0.035512
6172,bt,30,183,PD,0.042971


In [111]:
df_aguardar["dataset"].unique()

array(['fb2', 'muchas-soluciones-validas', 'pocas-soluciones-validas',
       'mejor-caso-bt-f', 'dinamica', 'bt'], dtype=object)

In [112]:
df_aguardar = df_aguardar.append(df_resultado,ignore_index = True)
df_aguardar

,dataset,n,R,metodo,tiempo
0,fb2,5,1,FB,0.002025
1,fb2,5,6,FB,0.001162
2,fb2,5,11,FB,0.001282
3,fb2,5,16,FB,0.001511
4,fb2,5,21,FB,0.001092
...,...,...,...,...,...
6217,pocas-soluciones-validas,22,1000,BT,0.002905
6218,pocas-soluciones-validas,23,1000,PD,0.033111
6219,pocas-soluciones-validas,23,1000,BT,0.003059
6220,pocas-soluciones-validas,24,1000,PD,0.034491


In [113]:
df_aguardar.to_csv("resultados/resultado.csv", index=False, header=True);